# Titulo proyecto

### Descripcion

## Limpieza datos

Explicar que se se tuvo que usar la solución definitiva: abandonaremos pd.read_csv() y leeremos el archivo como un simple archivo de texto, línea por línea, usando Python nativo. Luego, construiremos el DataFrame manualmente.

Este método es a prueba de errores de formato.

Solución Final: Lectura Manual del Archivo
Este código abre el archivo, lee cada línea una por una (ignorando la primera que es el encabezado) y las guarda en una lista. Finalmente, crea el DataFrame a partir de esa lista.

In [ ]:
# Importamos la librería principal
import pandas as pd
import os

# Definimos la ruta del archivo
ruta_archivo = 'data/datos_raw_inmuebles_santiago.csv'

# Creamos una lista vacía para guardar cada línea de datos
lista_de_datos = []

print("Intentando cargar el archivo línea por línea...")

try:
    # --- MÉTODO A PRUEBA DE ERRORES ---
    # Abrimos el archivo en modo lectura ('r') con la codificación correcta
    with open(ruta_archivo, 'r', encoding='utf-8-sig') as f:
        
        # Saltamos la primera línea, que es el encabezado 'datos_brutos'
        next(f)
        
        # Leemos cada línea restante del archivo
        for linea in f:
            # Añadimos la línea a nuestra lista, quitando espacios o saltos de línea
            # al principio y al final (.strip())
            lista_de_datos.append(linea.strip())

    # Creamos el DataFrame a partir de la lista que llenamos
    df = pd.DataFrame(lista_de_datos, columns=['datos_brutos'])

    # --- Verificación Final ---
    print("\n¡Éxito! El archivo se cargó correctamente.")
    print(f"Se cargaron {len(df)} registros.")
    df.head()
except FileNotFoundError:
    print(f"Error: No se pudo encontrar el archivo en la ruta '{ruta_archivo}'")
except Exception as e:
    print(f"Ocurrió un error inesperado durante la lectura del archivo: {e}")
df.head()

Intentando cargar el archivo línea por línea...

¡Éxito! El archivo se cargó correctamente.
Se cargaron 2016 registros.


,datos_brutos
0,PROYECTO | Departamentos en venta | Edificio C...
1,PROYECTO | Departamentos en venta | San Ignaci...
2,PROYECTO | Departamentos en venta | Sinergía V...
3,PROYECTO | Departamentos en venta | Jofre | UF...
4,PROYECTO | Departamentos en venta | Eco Espaci...


In [2]:
# Asumimos que tu DataFrame 'df' ya está cargado con la columna 'datos_brutos'

# --- PASO 1: Extraer Moneda y Precio (Versión Definitiva) ---

# Usamos un regex que busca el patrón "| UF | número |" o "| $ | número |"
# y captura la moneda y el número.
df[['Moneda', 'Precio_str']] = df['datos_brutos'].str.extract(r'\|\s*(UF|\$)\s*\|\s*([\d\.]+)', expand=True)

# El resto del código de limpieza se mantiene igual.
# Limpiamos la columna de precio: quitamos los puntos y la convertimos a tipo numérico.
df['Precio'] = pd.to_numeric(df['Precio_str'].str.replace('.', '', regex=False), errors='coerce')

# Asignamos el tipo Int64 que permite valores nulos (NaN)
df['Precio'] = df['Precio'].astype('Int64')

# Verificamos las primeras 5 filas de las columnas extraídas para confirmar el éxito
print("Verificación de Moneda y Precio extraídos:")
print(df[['Moneda', 'Precio']].head())


# --- PASO 2: Extraer Dormitorios ---
# Buscamos un número seguido de la palabra "dorm" o "dormitorio(s)".
# El regex (\d+) captura uno o más dígitos.
df['Dormitorios'] = df['datos_brutos'].str.extract(r'(\d+)\s*dorms?|(\d+)\s*dormitorio?s?').bfill(axis=1)[0]
# Convertimos a tipo numérico. Int64 (con mayúscula) permite valores nulos (NaN).
df['Dormitorios'] = pd.to_numeric(df['Dormitorios'], errors='coerce').astype('Int64')


# --- PASO 3: Extraer Baños ---
# Buscamos un número seguido de la palabra "baño(s)".
df['Banos'] = df['datos_brutos'].str.extract(r'(\d+)\s*baños?', expand=False)
# Convertimos a tipo numérico nullable.
df['Banos'] = pd.to_numeric(df['Banos'], errors='coerce').astype('Int64')


# --- PASO 4: Extraer Superficie ---
# Buscamos un número (puede tener decimales) seguido de "m²".
# Capturamos solo el primer número si hay un rango (ej. "22-31 m²").
df['Superficie_m2'] = df['datos_brutos'].str.extract(r'([\d\.]+)(?:-[\d\.]+)?\s*m²', expand=False)
# Convertimos a tipo numérico.
df['Superficie_m2'] = pd.to_numeric(df['Superficie_m2'], errors='coerce')


# --- PASO 5: Extraer Dirección y Comuna ---
# Este es más complejo. Asumimos una estructura como "Calle 123, Comuna, RM".
# El regex captura los dos textos separados por una coma antes de "RM".
df[['Direccion', 'Comuna']] = df['datos_brutos'].str.extract(r'([^,]+),\s*([^,]+),\s*RM', expand=True)
# Limpiamos espacios en blanco al inicio o final que pudieran quedar.
if 'Direccion' in df.columns:
    df['Direccion'] = df['Direccion'].str.strip()
if 'Comuna' in df.columns:
    df['Comuna'] = df['Comuna'].str.strip()


# --- PASO 6: Creación del DataFrame Final y Verificación ---

# Seleccionamos solo las columnas que creamos, en el orden deseado.
columnas_finales = ['Moneda', 'Precio', 'Dormitorios', 'Banos', 'Superficie_m2', 'Direccion', 'Comuna']
df_limpio = df[columnas_finales].copy()

# Usamos .info() para ver un resumen del DataFrame: las columnas, la cantidad de datos no nulos y el TIPO DE DATO.
print("--- Información del DataFrame Limpio ---")
df_limpio.info()

# Mostramos las primeras filas del resultado final.
print("\n--- Vista Previa del DataFrame Limpio ---")
df_limpio.head()

Verificación de Moneda y Precio extraídos:
  Moneda  Precio
0     UF    2516
1     UF    2245
2     UF    2160
3     UF    3092
4     UF    2630


C:\Users\fabri\AppData\Local\Temp\ipykernel_44152\2680123583.py:24: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Dormitorios'] = df['datos_brutos'].str.extract(r'(\d+)\s*dorms?|(\d+)\s*dormitorio?s?').bfill(axis=1)[0]


--- Información del DataFrame Limpio ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2016 entries, 0 to 2015
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Moneda         2015 non-null   object 
 1   Precio         2015 non-null   Int64  
 2   Dormitorios    2006 non-null   Int64  
 3   Banos          2010 non-null   Int64  
 4   Superficie_m2  2013 non-null   float64
 5   Direccion      2015 non-null   object 
 6   Comuna         2015 non-null   object 
dtypes: Int64(3), float64(1), object(3)
memory usage: 116.3+ KB

--- Vista Previa del DataFrame Limpio ---


,Moneda,Precio,Dormitorios,Banos,Superficie_m2,Direccion,Comuna
0,UF,2516,2,2,53.0,Bulnes,Santiago
1,UF,2245,2,2,52.0,Dieciocho,Santiago
2,UF,2160,2,2,62.0,Bogotá - Sierra Bella,Santiago
3,UF,3092,1,1,32.0,Santa Isabel,Santiago
4,UF,2630,2,2,63.0,Santa Isabel,Santiago


In [11]:
# Convierte el índice actual del DataFrame 'df_limpio' en una nueva columna llamada 'index'
df_limpio.reset_index(inplace=True)

# Renombramos esa nueva columna de 'index' a 'id' para que sea más claro
df_limpio.rename(columns={'index': 'id'}, inplace=True)

# Mostramos las primeras filas para verificar que la nueva columna 'id' fue creada correctamente
df_limpio.head()

,id,Moneda,Precio,Dormitorios,Banos,Superficie_m2,Direccion,Comuna
0,0,UF,2516,2,2,53.0,Bulnes,Santiago
1,1,UF,2245,2,2,52.0,Dieciocho,Santiago
2,2,UF,2160,2,2,62.0,Bogotá - Sierra Bella,Santiago
3,3,UF,3092,1,1,32.0,Santa Isabel,Santiago
4,4,UF,2630,2,2,63.0,Santa Isabel,Santiago


In [ ]:
# Establecemos la columna 'id' como el nuevo índice del DataFrame.
# El índice anterior (0, 1, 2...) será reemplazado.
df_limpio.set_index('id', inplace=True)

# Mostramos el resultado final. Verás que 'id' ahora ocupa el lugar del índice.
df_limpio.head()

,Moneda,Precio,Dormitorios,Banos,Superficie_m2,Direccion,Comuna
id,,,,,,,
0,UF,2516,2,2,53.0,Bulnes,Santiago
1,UF,2245,2,2,52.0,Dieciocho,Santiago
2,UF,2160,2,2,62.0,Bogotá - Sierra Bella,Santiago
3,UF,3092,1,1,32.0,Santa Isabel,Santiago
4,UF,2630,2,2,63.0,Santa Isabel,Santiago


In [18]:
import os # Asegúrate de que esta librería esté importada al inicio de tu notebook

# --- Guardar el DataFrame Limpio ---

# 1. Definir la carpeta y el nombre del archivo final
carpeta_salida = 'data'
nombre_archivo = 'datos_limpios_inmuebles.csv'
ruta_completa = os.path.join(carpeta_salida, nombre_archivo)

# 2. Crear la carpeta 'data' si no existe para evitar errores
os.makedirs(carpeta_salida, exist_ok=True)

# 3. Guardar el DataFrame en la ruta especificada
# El DataFrame se guarda con el índice, que ahora es tu columna 'id'.
df_limpio.to_csv(ruta_completa, encoding='utf-8-sig')

print(f"DataFrame guardado exitosamente en: {ruta_completa}")

DataFrame guardado exitosamente en: data\datos_limpios_inmuebles.csv


Como no queremos hacer un analisis geográfico, eliminaremos las coulmnas direccion y comuna

In [19]:
# Lista de las columnas que ya no necesitamos
columnas_a_eliminar = ['Direccion', 'Comuna']

# Usamos el método .drop() para eliminarlas del DataFrame
# axis=1 le indica a Pandas que estamos eliminando columnas
# inplace=True modifica df_limpio directamente sin necesidad de reasignarlo
df_limpio.drop(columns=columnas_a_eliminar, axis=1, inplace=True)

# Verificamos el resultado mostrando las primeras 5 filas.
# Las columnas 'Direccion' y 'Comuna' ya no deberían aparecer.
df_limpio.head()

,Moneda,Precio,Dormitorios,Banos,Superficie_m2
id,,,,,
0,UF,2516,2,2,53.0
1,UF,2245,2,2,52.0
2,UF,2160,2,2,62.0
3,UF,3092,1,1,32.0
4,UF,2630,2,2,63.0


In [21]:
# Contamos la frecuencia de cada valor en la columna 'Moneda'
conteo_moneda = df_limpio['Moneda'].value_counts()

# Imprimimos el resultado
print(conteo_moneda)

Moneda
UF    1785
$      230
Name: count, dtype: int64


Convertirimos los registros que estan en $ a UF usando el valor de la UF = 39209.42 al 26 de julio del 2025 y redondenando el resultado.

In [22]:
# 1. Definimos el valor de la UF que usaremos para la conversión
valor_uf = 39209.42

# 2. Seleccionamos las filas que tienen precios en pesos chilenos ('$')
# Esto crea una máscara booleana (True/False) para identificar las filas a convertir
filas_a_convertir = df_limpio['Moneda'] == '$'

# 3. Aplicamos la conversión solo a las filas seleccionadas
# Dividimos el 'Precio' por el valor de la UF y lo redondeamos
df_limpio.loc[filas_a_convertir, 'Precio'] = (df_limpio.loc[filas_a_convertir, 'Precio'] / valor_uf).round()

# 4. Actualizamos la columna 'Moneda' a 'UF' en las filas que acabamos de convertir
df_limpio.loc[filas_a_convertir, 'Moneda'] = 'UF'

# 5. Aseguramos que la columna 'Precio' mantenga el tipo de dato correcto (entero)
# El tipo 'Int64' (con I mayúscula) permite números enteros y valores nulos (NaN)
df_limpio['Precio'] = df_limpio['Precio'].astype('Int64')

# --- Verificación ---
print("--- Verificación de Moneda después de la conversión ---")
# Ahora solo debería aparecer 'UF' (y NaN si hay valores nulos)
print(df_limpio['Moneda'].value_counts(dropna=False))

print("\n--- Vista Previa del DataFrame con precios unificados ---")
df_limpio.head()

--- Verificación de Moneda después de la conversión ---
Moneda
UF     2015
NaN       1
Name: count, dtype: int64

--- Vista Previa del DataFrame con precios unificados ---


,Moneda,Precio,Dormitorios,Banos,Superficie_m2
id,,,,,
0,UF,2516,2,2,53.0
1,UF,2245,2,2,52.0
2,UF,2160,2,2,62.0
3,UF,3092,1,1,32.0
4,UF,2630,2,2,63.0


In [24]:
df_limpio.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2016 entries, 0 to 2015
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Moneda         2015 non-null   object 
 1   Precio         2015 non-null   Int64  
 2   Dormitorios    2006 non-null   Int64  
 3   Banos          2010 non-null   Int64  
 4   Superficie_m2  2013 non-null   float64
dtypes: Int64(3), float64(1), object(1)
memory usage: 100.4+ KB


In [25]:
df_limpio.isnull().sum()

Moneda            1
Precio            1
Dormitorios      10
Banos             6
Superficie_m2     3
dtype: int64

Para tratar con los nulos en este caso no los eliminaremos porque queremos usar todos los datos extraidos y NO usaremos el valor promedio ya que en general, no es la mejor idea para este tipo de datos, y te explico por qué.

¿Por Qué no es lo Ideal?
No tiene sentido para algunas columnas: Para Dormitorios y Banos, el promedio podría ser, por ejemplo, 2.3, y no existen departamentos con "2.3 baños". Necesitas un número entero.

Sensibilidad a valores atípicos: En el mercado inmobiliario, unas pocas propiedades extremadamente caras pueden "inflar" el promedio. Si usas ese promedio para rellenar los nulos, estarías asignando un valor artificialmente alto a propiedades de las que no conoces el precio.

En general, no es la mejor idea para este tipo de datos, y te explico por qué.

¿Por Qué no es lo Ideal?
No tiene sentido para algunas columnas: Para Dormitorios y Banos, el promedio podría ser, por ejemplo, 2.3, y no existen departamentos con "2.3 baños". Necesitas un número entero.

Sensibilidad a valores atípicos: En el mercado inmobiliario, unas pocas propiedades extremadamente caras pueden "inflar" el promedio. Si usas ese promedio para rellenar los nulos, estarías asignando un valor artificialmente alto a propiedades de las que no conoces el precio.

Alternativas Recomendadas (Mejores Prácticas)
Aquí hay dos enfoques que son mucho más robustos y profesionalmente aceptados para este caso.

1. Usar la Mediana (La opción más segura)
La mediana es el valor que se encuentra justo en el medio de todos tus datos si los ordenaras de menor a mayor. A diferencia del promedio, no se ve afectada por valores extremos.

Para Precio y Superficie_m2: Es ideal porque ignora las mansiones o los departamentos extremadamente caros que desvían el promedio.

Para Dormitorios y Banos: Te dará un número entero y común (ej. 2), lo cual tiene mucho más sentido.

In [27]:
# Hacemos una copia del DataFrame para trabajar sobre ella y mantener el original si es necesario
df_final = df_limpio.copy()

# Lista de columnas numéricas en las que queremos rellenar los nulos
columnas_numericas = ['Precio', 'Dormitorios', 'Banos', 'Superficie_m2']

# Iteramos sobre cada columna de la lista
for columna in columnas_numericas:
    # 1. Calculamos la mediana de la columna actual
    mediana = df_final[columna].median()
    
    # 2. Rellenamos los valores nulos (NaN) de esa columna con la mediana calculada
    df_final[columna].fillna(mediana, inplace=True)
    
    # 3. Opcional: convertimos las columnas que deben ser enteras de vuelta a tipo entero
    if columna in ['Precio', 'Dormitorios', 'Banos']:
        # Ahora que no hay nulos, podemos usar 'int' de forma segura
        df_final[columna] = df_final[columna].astype(int)

# --- Verificación ---
# Comprobamos de nuevo el conteo de nulos. Las columnas numéricas deberían aparecer con 0.
print("--- Conteo de Nulos Después de Rellenar ---")
print(df_final.isnull().sum())

print("\n--- Vista Previa del DataFrame Final ---")
df_final.head()

--- Conteo de Nulos Después de Rellenar ---
Moneda           1
Precio           0
Dormitorios      0
Banos            0
Superficie_m2    0
dtype: int64

--- Vista Previa del DataFrame Final ---


C:\Users\fabri\AppData\Local\Temp\ipykernel_44152\2375518189.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_final[columna].fillna(mediana, inplace=True)
C:\Users\fabri\AppData\Local\Temp\ipykernel_44152\2375518189.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For ex

,Moneda,Precio,Dormitorios,Banos,Superficie_m2
id,,,,,
0,UF,2516,2,2,53.0
1,UF,2245,2,2,52.0
2,UF,2160,2,2,62.0
3,UF,3092,1,1,32.0
4,UF,2630,2,2,63.0


Eliminamos por ultimoa la columna Moneda para solo trabajar con datos numéricos.

In [28]:
# Eliminamos la columna 'Moneda' del DataFrame
# axis=1 especifica que estamos eliminando una columna
# inplace=True modifica df_final directamente
df_final.drop('Moneda', axis=1, inplace=True)

# Verificamos el resultado mostrando las primeras filas del DataFrame actualizado
df_final.head()

,Precio,Dormitorios,Banos,Superficie_m2
id,,,,
0,2516,2,2,53.0
1,2245,2,2,52.0
2,2160,2,2,62.0
3,3092,1,1,32.0
4,2630,2,2,63.0
